In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import warnings
import pkg_resources
warnings.filterwarnings('ignore')

# Configuration
TFLITE_DIR = '../models/models_approach1/tflite'
MODEL_PATH = os.path.join(TFLITE_DIR, 'model_logistic_regression_dynamic_quant.tflite')
OUTPUT_PATH = os.path.join(TFLITE_DIR, 'model_with_metadata.tflite')

# Model information
MODEL_INFO = {
    "name": "YAMNet Audio Event Classifier",
    "version": "1.0.0",
    "description": "Audio event detection model using YAMNet embeddings and Logistic Regression classifier. Detects: Alarm_Clock, Car_Horn, Glass_Breaking, Gunshot, Siren.",
    "license": "Apache-2.0",
    "classes": ["Alarm_Clock", "Car_Horn", "Glass_Breaking", "Gunshot", "Siren"],
    "sample_rate": 16000,
    "input_duration_seconds": 0.96,
    "input_samples": 15360,
    "input_shape": [1, 15360],
    "input_dtype": "float32",
    "output_shape": [1, 5],
    "output_dtype": "float32",
    "accuracy": 0.9110,
    "f1_score": 0.9109,
    "confidence_threshold": 0.6,
    "tflite_version": "2.20.0"
}

def check_environment():
    """Check TensorFlow, tflite-support, and flatbuffers versions"""
    try:
        import tensorflow as tf
        from tflite_support import metadata as _metadata
        import flatbuffers
        tf_version = tf.__version__
        tflite_support_version = pkg_resources.get_distribution("tflite-support").version
        flatbuffers_version = getattr(flatbuffers, '__version__', 'unknown')
        print(f"\n📦 TensorFlow Version: {tf_version}")
        print(f"📦 tflite-support Version: {tflite_support_version}")
        print(f"📦 flatbuffers Version: {flatbuffers_version}")
        if tflite_support_version.startswith('0.1.'):
            print("\n⚠️ tflite-support version 0.1.x detected. Limited functionality.")
            print("   Strongly recommend upgrading: pip install tflite-support==0.4.4")
        if int(tf_version.split('.')[0]) < 2 or (int(tf_version.split('.')[0]) == 2 and int(tf_version.split('.')[1]) < 6):
            print("\n⚠️ TensorFlow version is too old. Minimum required: 2.6.0")
            print("   Upgrade with: pip install --upgrade tensorflow")
            return False
        return True
    except ImportError as e:
        print(f"\n❌ Missing required package: {e}")
        print("   Install with: pip install tensorflow tflite-support flatbuffers")
        return False
    except Exception as e:
        print(f"\n❌ Environment check failed: {e}")
        return False

def verify_model_structure():
    """Verify the model structure before embedding metadata"""
    if not os.path.exists(MODEL_PATH):
        print(f"\n❌ Model file not found: {MODEL_PATH}")
        print("   Ensure the model file exists at the specified path.")
        return False

    try:
        interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
        interpreter.allocate_tensors()

        print("\n" + "="*70)
        print("MODEL STRUCTURE VERIFICATION")
        print("="*70)

        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        print(f"\n📥 INPUT TENSORS: {len(input_details)}")
        for i, detail in enumerate(input_details):
            print(f"  Input {i}: {detail['name']} - Shape: {detail['shape']} - Type: {detail['dtype']}")

        print(f"\n📤 OUTPUT TENSORS: {len(output_details)}")
        for i, detail in enumerate(output_details):
            print(f"  Output {i}: {detail['name']} - Shape: {detail['shape']} - Type: {detail['dtype']}")

        print("\n✅ Model verification complete")
        print("="*70)
        return True

    except Exception as e:
        print(f"\n⚠️ Error verifying model: {e}")
        print("\n💡 Possible causes:")
        print("   - Model was created with a newer TensorFlow version than 2.20.0.")
        print("   - Model contains unsupported operations.")
        print("   Try the following:")
        print("   1. Update TensorFlow: pip install --upgrade tensorflow")
        print("   2. Reconvert the model using TensorFlow 2.20.0.")
        print("   3. Install tflite-runtime: pip install tflite-runtime")
        print("   4. Check for custom operations in the model.")
        return False

def embed_metadata_into_model():
    """Embed comprehensive metadata directly into the TFLite model"""
    try:
        print("\n" + "="*70)
        print("EMBEDDING METADATA INTO MODEL")
        print("="*70)

        from tflite_support import metadata as _metadata
        from tflite_support import metadata_schema_py_generated as _metadata_fb
        import flatbuffers

        print("\n📝 Building metadata structure...")

        model_meta = _metadata_fb.ModelMetadataT()
        model_meta.name = MODEL_INFO["name"]
        model_meta.description = (
            f"{MODEL_INFO['description']} "
            f"Accuracy: {MODEL_INFO['accuracy']:.2%}, F1: {MODEL_INFO['f1_score']:.4f}. "
            f"Designed for TensorFlow Lite {MODEL_INFO['tflite_version']}. "
            f"Mobile app integration: Preprocess audio to {MODEL_INFO['sample_rate']}Hz, "
            f"mono, {MODEL_INFO['input_samples']} samples, normalized [-1.0, 1.0]. "
            f"Output probabilities: Apply argmax for class index, use confidence threshold "
            f">{MODEL_INFO['confidence_threshold']} to reduce false positives."
        )
        model_meta.version = MODEL_INFO["version"]
        model_meta.author = MODEL_INFO["author"]
        model_meta.license = MODEL_INFO["license"]

        input_meta = _metadata_fb.TensorMetadataT()
        input_meta.name = "input_tensor"
        input_meta.description = (
            f"Input tensor expecting shape {MODEL_INFO['input_shape']}, "
            f"dtype {MODEL_INFO['input_dtype']}, mono audio at {MODEL_INFO['sample_rate']}Hz, "
            f"{MODEL_INFO['input_duration_seconds']}s duration, normalized to [-1.0, 1.0]. "
            f"Preprocessing: Resample audio to {MODEL_INFO['sample_rate']}Hz, convert to mono, "
            f"normalize samples to [-1.0, 1.0], ensure exactly {MODEL_INFO['input_samples']} samples."
        )
        input_meta.content = _metadata_fb.ContentT()

        output_metas = []
        output_meta = _metadata_fb.TensorMetadataT()
        output_meta.name = "class_probabilities"
        output_meta.description = (
            f"Output tensor with shape {MODEL_INFO['output_shape']}, "
            f"dtype {MODEL_INFO['output_dtype']}, representing probability distribution "
            f"over {len(MODEL_INFO['classes'])} classes: {', '.join(MODEL_INFO['classes'])}. "
            f"Post-processing: Use argmax to get predicted class index (0 to {len(MODEL_INFO['classes'])-1}). "
            f"Apply confidence threshold >{MODEL_INFO['confidence_threshold']} to filter predictions."
        )
        stats = _metadata_fb.StatsT()
        stats.max = [1.0] * len(MODEL_INFO["classes"])
        stats.min = [0.0] * len(MODEL_INFO["classes"])
        output_meta.stats = stats
        output_metas.append(output_meta)

        subgraph = _metadata_fb.SubGraphMetadataT()
        subgraph.name = "Classifier"
        subgraph.description = (
            f"Audio classification subgraph. "
            f"Input: {MODEL_INFO['input_shape']} audio samples at {MODEL_INFO['sample_rate']}Hz. "
            f"Output: {MODEL_INFO['output_shape']} probabilities over {len(MODEL_INFO['classes'])} classes."
        )
        subgraph.inputTensorMetadata = [input_meta]
        subgraph.outputTensorMetadata = output_metas
        model_meta.subgraphMetadata = [subgraph]

        print("🔨 Compiling metadata flatbuffer...")
        b = flatbuffers.Builder(0)
        b.Finish(
            model_meta.Pack(b),
            _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER
        )
        metadata_buf = b.Output()

        print(f"📦 Metadata buffer size: {len(metadata_buf)} bytes")

        print("🔧 Embedding metadata into model...")
        with open(MODEL_PATH, 'rb') as f:
            model_buffer = f.read()

        populator = _metadata.MetadataPopulator.with_model_buffer(model_buffer)
        populator.load_metadata_buffer(metadata_buf)
        populator.populate()

        with open(OUTPUT_PATH, 'wb') as f:
            f.write(populator.get_model_buffer())

        output_size = os.path.getsize(OUTPUT_PATH) / 1024
        print(f"\n✅ Metadata successfully embedded!")
        print(f"📁 Output model: {OUTPUT_PATH}")
        print(f"📊 Output size: {output_size:.2f} KB")

        return True

    except Exception as e:
        print(f"\n❌ Error embedding metadata: {e}")
        print("\n💡 Troubleshooting steps:")
        print("   1. Upgrade tflite-support: pip install tflite-support==0.4.4")
        print("   2. Verify model compatibility with TensorFlow 2.20.0.")
        print("   3. Check write permissions: icacls \"" + TFLITE_DIR + "\" /grant:r \"%username%:F\" /t (Windows)")
        print("   4. Ensure the model file is not corrupted.")
        import traceback
        traceback.print_exc()
        return False

def display_embedded_metadata():
    """Display the metadata embedded in the model"""
    try:
        from tflite_support import metadata as _metadata

        print("\n" + "="*70)
        print("EMBEDDED METADATA CONTENT")
        print("="*70)

        displayer = _metadata.MetadataDisplayer.with_model_file(OUTPUT_PATH)

        print("\n📄 JSON Metadata:")
        print("-" * 70)
        metadata_json = displayer.get_metadata_json()
        if metadata_json:
            import json
            metadata_dict = json.loads(metadata_json)
            print(json.dumps(metadata_dict, indent=2))
        else:
            print("No metadata found")

        print("\n" + "="*70)

    except ImportError:
        print("\n⚠️ tflite-support not available for metadata display")
        print("   Install with: pip install tflite-support")
    except Exception as e:
        print(f"\n⚠️ Could not display metadata: {e}")

def main():
    """Main execution"""
    print("=" * 70)
    print("🚀 TFLITE MODEL METADATA EMBEDDER")
    print("=" * 70)
    print("\nThis script embeds comprehensive metadata into the .tflite model for mobile app integration")

    print("\n📝 Step 0/3: Checking environment...")
    if not check_environment():
        print("\n❌ Environment check failed. Aborting.")
        print("   Run: pip install --upgrade tensorflow tflite-support flatbuffers")
        return

    print("\n📝 Step 1/3: Verifying model structure...")
    if not verify_model_structure():
        print("\n⚠️ Model verification failed. Attempting to proceed...")
        print("   If metadata embedding fails, reconvert the model with TensorFlow 2.20.0.")
        print("   Warning: Model may fail during inference due to unsupported operations.")

    print("\n📝 Step 2/3: Embedding metadata into model...")
    if not embed_metadata_into_model():
        print("\n❌ Metadata embedding failed.")
        return

    print("\n📝 Step 3/3: Displaying embedded metadata...")
    display_embedded_metadata()

    print("\n" + "="*70)
    print("✨ PROCESS COMPLETE")
    print("="*70)
    print(f"\n✅ Model with metadata saved at: {OUTPUT_PATH}")
    print("\n📋 Embedded metadata includes:")
    print("  ✓ Model name, version, description")
    print("  ✓ Author and license")
    print("  ✓ Input specifications (shape, dtype, preprocessing)")
    print("  ✓ Output specifications (shape, dtype, post-processing)")
    print("  ✓ Mobile app integration instructions")
    print("\n🎯 Key Information for Mobile App:")
    print(f"  • Classes: {', '.join(MODEL_INFO['classes'])}")
    print(f"  • Input: {MODEL_INFO['input_shape']} {MODEL_INFO['input_dtype']} audio samples")
    print(f"  • Output: {MODEL_INFO['output_shape']} {MODEL_INFO['output_dtype']} probabilities")
    print(f"  • Preprocessing: Resample to {MODEL_INFO['sample_rate']}Hz, mono, normalize to [-1.0, 1.0]")
    print(f"  • Post-processing: Apply argmax, use confidence threshold >{MODEL_INFO['confidence_threshold']}")
    print(f"  • Accuracy: {MODEL_INFO['accuracy']:.2%}")
    print(f"  • F1 Score: {MODEL_INFO['f1_score']:.4f}")
    print("\n⚠️ Note: Model verification failed. Ensure compatibility with TensorFlow Lite runtime for mobile inference.")

if __name__ == "__main__":
    main()

🚀 TFLITE MODEL METADATA EMBEDDER

This script embeds comprehensive metadata into the .tflite model for mobile app integration

📝 Step 0/3: Checking environment...

📦 TensorFlow Version: 2.20.0
📦 tflite-support Version: 0.1.0a1
📦 flatbuffers Version: 25.9.23

⚠️ tflite-support version 0.1.x detected. Limited functionality.
   Strongly recommend upgrading: pip install tflite-support==0.4.4

📝 Step 1/3: Verifying model structure...

⚠️ Error verifying model: Didn't find op for builtin opcode 'PLACEHOLDER_FOR_GREATER_OP_CODES' version '1'. An older version of this builtin might be supported. Are you using an old TFLite binary with a newer model?
Registration failed.


💡 Possible causes:
   - Model was created with a newer TensorFlow version than 2.20.0.
   - Model contains unsupported operations.
   Try the following:
   1. Update TensorFlow: pip install --upgrade tensorflow
   2. Reconvert the model using TensorFlow 2.20.0.
   3. Install tflite-runtime: pip install tflite-runtime
   4. Ch